# AOC 2021

Welcome to the Advent of Code 2021 !

## Basic configuration



In [ ]:
# help for aocd : https://pypi.org/project/advent-of-code-data/

#!pip install aocd

In [3]:
import os

# replace by your login session cookie
os.environ[
    "AOC_SESSION"
] = " "  # your login session cookie

In [4]:
from aocd import submit
from aocd.models import Puzzle

In [5]:
import numpy as np
from tqdm import tqdm
import json
import typing as tp
from collections import Counter, defaultdict, deque
import math
from itertools import product
import re
import string
import matplotlib.pyplot as plt

## Day 24
https://adventofcode.com/2021/day/24
### Prepare input

In [ ]:
puzzle = Puzzle(year=2021, day=24)

### Part 1

### Part 2

## Day 23
https://adventofcode.com/2021/day/23
### Prepare input

In [ ]:
puzzle = Puzzle(year=2021, day=23)

### Part 1

### Part 2

## Day 22
https://adventofcode.com/2021/day/22
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2021, day=22)

### Part 1

### Part 2

## Day 21
https://adventofcode.com/2021/day/21
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2021, day=21)

### Part 1

### Part 2

## Day 20
https://adventofcode.com/2021/day/20
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2021, day=20)
algorithm, content = puzzle.input_data.split("\n\n")

In [ ]:
content = content.split("\n")

In [ ]:
def set_image(content: str) -> defaultdict:
    image = defaultdict(lambda: 0)

    for y, line in enumerate(content):
        for x, char in enumerate(line):
            image[(y, x)] = 1 if char == "#" else 0
    return image


### Part 1

In [ ]:
Coord = tp.Tuple[int, int]

In [ ]:
def get_adjacents(coord: Coord) -> tp.List[Coord]:
    return [(coord[0] + dy, coord[1] + dx) for dy, dx in product([-1, 0, 1], repeat=2)]

In [ ]:
get_adjacents((0, 0))  # je me fatigue

In [ ]:
def get_value(image: defaultdict, coord: Coord) -> int:
    window = get_adjacents(coord)

    binary = "".join([str(image[coord]) for coord in window])
    return int(binary, 2)

In [ ]:
def enhance(image: defaultdict, default: int) -> defaultdict:

    new_image = defaultdict(lambda: default)

    ymin = min(image.keys(), key=lambda coord: coord[0])[0]
    ymax = max(image.keys(), key=lambda coord: coord[0])[0]

    xmin = min(image.keys(), key=lambda coord: coord[1])[1]
    xmax = max(image.keys(), key=lambda coord: coord[1])[1]

    for y in range(ymin - 1, ymax + 2):
        for x in range(xmin - 1, xmax + 2):
            coord = (y, x)
            value = get_value(image, coord)
            new_image[coord] = 1 if algorithm[value] == "#" else 0

    return new_image

In [ ]:
image = set_image(content)

In [ ]:
for idx in range(2):
    if algorithm[0] == "#" and algorithm[-1] == ".":
        default = (idx + 1) % 2
    elif algorithm[0] == "#" and algorithm[-1] == "#":
        default = 1
    else:
        default = 0

    image = enhance(image, default)

In [ ]:
count = Counter(image.values())

In [ ]:
count[1]

In [ ]:
puzzle.answer_a = count[1]

### Part 2

In [ ]:
image = set_image(content)

for idx in range(50):
    if algorithm[0] == "#" and algorithm[-1] == ".":
        default = (idx + 1) % 2
    elif algorithm[0] == "#" and algorithm[-1] == "#":
        default = 1
    else:
        default = 0

    image = enhance(image, default)

In [ ]:
count = Counter(image.values())
count[1]

In [ ]:
puzzle.answer_b = count[1]

## Day 19
https://adventofcode.com/2021/day/19
### Prepare input

In [ ]:
puzzle = Puzzle(year=2021, day=19)
content = puzzle.input_data.split("\n\n")

In [ ]:
content = [line.split("\n") for line in content]

### Part 1

In [ ]:
from scipy.spatial.transform import Rotation as R

euler_matrices = [
    [
        R.from_euler(axis, degree, degrees=True).as_matrix().astype(int)
        for degree in {0, 90, 180, 270}
    ]
    for axis in {"x", "y", "z"}
]

ROT_MATRICES = []
for rx in euler_matrices[0]:
    for ry in euler_matrices[1]:
        for rz in euler_matrices[2]:
            rot = np.matmul(np.matmul(rz, ry), rx)
            if not any(np.all(rot == mat) for mat in ROT_MATRICES):
                ROT_MATRICES.append(rot)

In [ ]:
def convert_to_set(array: np.array) -> tp.Set:
    return set(tuple(coord) for coord in array)

In [ ]:
class Scanner:
    def __init__(self, in_scanner: str) -> None:
        self.index = int(re.findall("-?\d+", in_scanner[0])[0])
        self.beacons = self.get_beacons(in_scanner[1:])
        self.rotations = self.get_all_rotations() if self.index != 0 else None
        self.coords = None

    @staticmethod
    def get_beacons(in_beacons: str) -> np.array:
        coords = []
        for coord in in_beacons:
            coords.append(list(map(int, coord.split(","))))
        return np.array(coords)

    def get_all_rotations(self) -> tp.List[np.array]:
        return [self.beacons.dot(rot) for rot in ROT_MATRICES]

    def set_coords(self, coords: tp.Tuple[int, int, int]) -> None:
        self.coords = coords

    @staticmethod
    def find_distance(beacons: np.array, reference: np.array) -> tp.Optional[np.array]:
        ref_coords = convert_to_set(reference)
        for beacon in beacons:
            for ref_beacon in reference:
                diff = beacon - ref_beacon
                new_coords = convert_to_set(beacons - diff)

                if len(new_coords & ref_coords) >= 12:
                    return diff

    def get_relative_position(
        self, reference: np.array
    ) -> tp.Optional[tp.Tuple[np.array, np.array]]:
        for rotation in self.rotations:
            rel_pos = self.find_distance(rotation, reference)
            if rel_pos is not None:
                return rotation - rel_pos, rel_pos
        return None, None

In [ ]:
scanners = [Scanner(line) for line in content]

### Part 1

In [ ]:
ref_scanner = scanners[0]

seen = {ref_scanner.index}
ref_scanner.set_coords((0, 0, 0))

reference = ref_scanner.beacons
while len(seen) != len(scanners):

    for scanner in scanners:
        if scanner.index in seen:
            continue
        match, relative_pos = scanner.get_relative_position(reference)
        if match is not None:
            print(f"Matched scanner {scanner.index}")
            scanner.set_coords(relative_pos)
            seen.add(scanner.index)

            combine = list(convert_to_set(reference) | convert_to_set(match))
            reference = np.array(combine)
            break

In [ ]:
answ = len(reference)
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

manhattan_dist = the sum of the absolute differences of the Cartesian coordinates


In [ ]:
man_dist = [
    np.sum(np.abs(np.array(scanner1.coords) - np.array(scanner2.coords)))
    for scanner1 in scanners
    for scanner2 in scanners
]

answ = max(man_dist)
answ

In [ ]:
puzzle.answer_b = answ

## Day 18
https://adventofcode.com/2021/day/18
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2021, day=18)
content = [json.loads(line) for line in puzzle.input_data.split("\n")]

### Part 1

In [ ]:
SnailNumber = tp.List[tp.Union[int, tp.List]]
Transformed = tp.List[tp.Tuple[int, int]]


def transform(number: SnailNumber, depth: int = 1) -> Transformed:
    left, right = number
    left = [(left, depth)] if isinstance(left, int) else transform(left, depth + 1)
    right = [(right, depth)] if isinstance(right, int) else transform(right, depth + 1)
    return left + right

In [ ]:
def undo_transform(snail_nb: Transformed) -> SnailNumber:
    while len(snail_nb) > 1:
        for idx, (cur_pair, next_pair) in enumerate(zip(snail_nb[:-1], snail_nb[1:])):
            cur_value, cur_depth = cur_pair
            next_value, next_depth = next_pair
            if cur_depth == next_depth:
                snail_nb = (
                    snail_nb[:idx]
                    + [([cur_value, next_value], cur_depth - 1)]
                    + snail_nb[idx + 2 :]
                )

                break

    return snail_nb[0][0]

-----------------

In [ ]:
def explode_number(snail_nb: Transformed) -> tp.Optional[Transformed]:
    for idx, pair in enumerate(snail_nb):
        value, depth = pair
        if depth == 5:
            if idx > 0:  # check left
                snail_nb[idx - 1] = (snail_nb[idx - 1][0] + value, snail_nb[idx - 1][1])
            if idx < len(snail_nb) - 2:  # check right
                snail_nb[idx + 2] = (
                    snail_nb[idx + 1][0] + snail_nb[idx + 2][0],
                    snail_nb[idx + 2][1],
                )

            snail_nb[idx : idx + 2] = [(0, depth - 1)]
            return snail_nb

In [ ]:
def split_number(snail_nb: Transformed) -> tp.Optional[Transformed]:
    for idx, pair in enumerate(snail_nb):
        value, depth = pair
        if value >= 10:
            snail_nb[idx : idx + 1] = [
                (value // 2, depth + 1),
                (math.ceil(value / 2), depth + 1),
            ]
            return snail_nb

In [ ]:
def add_numbers(number: SnailNumber, to_add: SnailNumber) -> Transformed:
    return transform([number, to_add])

In [ ]:
def reduce(number: Transformed) -> SnailNumber:

    while True:

        exploded = explode_number(number)

        if exploded is not None:
            number = exploded
            continue

        splitted = split_number(number)
        if splitted is not None:
            number = splitted
            continue

        return undo_transform(number)

In [ ]:
def add_all_numbers(content: tp.List[SnailNumber]) -> SnailNumber:
    snail_number = content[0]
    for line in content[1:]:
        snail_number = add_numbers(snail_number, line)
        snail_number = reduce(snail_number)
    return snail_number

In [ ]:
final_number = add_all_numbers(content)

In [ ]:
def magnitude(snail_nb: SnailNumber) -> int:
    if isinstance(snail_nb, int):
        return snail_nb
    return 3 * magnitude(snail_nb[0]) + 2 * magnitude(snail_nb[1])

In [ ]:
magn = magnitude(final_number)
magn

In [ ]:
puzzle.answer_a = magn


### Part 2

In [ ]:
def get_pairs(snail_nb: SnailNumber) -> tp.List[tp.List[SnailNumber]]:
    return [[sn1, sn2] for sn1 in snail_nb for sn2 in snail_nb if sn1 != sn2]

In [ ]:
pairs = get_pairs(content)

In [ ]:
magnitudes = [magnitude(add_all_numbers(pair)) for pair in pairs]

In [ ]:
answ = max(magnitudes)
answ

In [ ]:
puzzle.answer_b = answ

## Day 17
https://adventofcode.com/2021/day/17
### Prepare input

In [ ]:
puzzle = Puzzle(year=2021, day=17)
content = puzzle.input_data
content

In [ ]:
tar_x1, tar_x2, tar_y1, tar_y2 = map(int, re.findall("-?\d+", content))

### Part 1

In [ ]:
answ = tar_y1 * (tar_y1 + 1) // 2
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
def one_step(pos_x: int, pos_y: int, vx: int, vy: int) -> int:
    if pos_x > tar_x2 or pos_y < tar_y1:
        return 0

    if tar_x1 <= pos_x <= tar_x2 and tar_y1 <= pos_y <= tar_y2:
        return 1

    pos_x += vx
    pos_y += vy
    vx -= int(vx > 0)
    vy -= 1

    return one_step(pos_x, pos_y, vx, vy)

In [ ]:
n_hits = 0
for vx in range(0, tar_x2 + 1):
    for vy in range(tar_y1, -tar_y1):
        n_hits += one_step(0, 0, vx, vy)
n_hits

In [ ]:
puzzle.answer_b = n_hits

## Day 16
https://adventofcode.com/2021/day/16
### Prepare input

In [ ]:
puzzle = Puzzle(year=2021, day=16)
content = puzzle.input_data

### Part 1

In [ ]:
def hex2bin(hexa: str) -> str:
    return bin(int(hexa, 16))[2:].zfill(len(hexa) * 4)

In [ ]:
def get_version_type(stream: str) -> tp.Tuple[int, int, str]:
    return int(stream[:3], 2), int(stream[3:6], 2), stream[6:]

In [ ]:
def get_type4_packet(stream: str) -> tp.Tuple[str, int]:
    packet = ""
    for cur_id in range(0, len(stream), 5):
        packet += stream[cur_id + 1 : cur_id + 5]
        if stream[cur_id] == "0":
            return int(packet, 2), stream[cur_id + 5 :]

In [ ]:
def get_len_typeid(stream: str) -> tp.Tuple[str, str]:
    return int(stream[0], 2), stream[1:]

In [ ]:
def parse_packet(bin_packet: str) -> tp.Tuple[tp.List[int], str]:

    version, type_id, bin_packet = get_version_type(bin_packet)

    versions = [version]

    if type_id == 4:
        packet_value, bin_packet = get_type4_packet(bin_packet)

    else:
        length_type_id, bin_packet = get_len_typeid(bin_packet)

        if length_type_id == 0:
            len_subs = int(bin_packet[:15], 2)
            bin_packet = bin_packet[15:]

            sub_packet = bin_packet[:len_subs]
            while sub_packet:
                subversions, sub_packet = parse_packet(sub_packet)
                versions.extend(subversions)

            bin_packet = bin_packet[len_subs:]

        else:
            n_subs = int(bin_packet[:11], 2)
            bin_packet = bin_packet[11:]

            for idx in range(n_subs):
                subversions, bin_packet = parse_packet(bin_packet)
                versions.extend(subversions)

    return versions, bin_packet

In [ ]:
binary = hex2bin(content)

In [ ]:
versions, _ = parse_packet(binary)

In [ ]:
answ = sum(versions)
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
def operation(type_id: int, values: tp.Sequence[int]) -> int:
    if type_id == 0:
        return np.sum(values)
    if type_id == 1:
        return np.prod(values)
    if type_id == 2:
        return np.min(values)
    if type_id == 3:
        return np.max(values)
    if type_id == 5:
        return int(values[0] > values[1])
    if type_id == 6:
        return int(values[0] < values[1])
    if type_id == 7:
        return int(values[0] == values[1])

In [ ]:
def parse_packet_2(bin_packet: str) -> tp.Tuple[int, str]:

    version, type_id, bin_packet = get_version_type(bin_packet)

    values = []

    if type_id == 4:
        packet_value, bin_packet = get_type4_packet(bin_packet)
        return packet_value, bin_packet

    else:
        length_type_id, bin_packet = get_len_typeid(bin_packet)

        if length_type_id == 0:
            len_subs = int(bin_packet[:15], 2)
            bin_packet = bin_packet[15:]

            sub_packet = bin_packet[:len_subs]
            while sub_packet:
                sub_value, sub_packet = parse_packet_2(sub_packet)
                values.append(sub_value)

            bin_packet = bin_packet[len_subs:]

        else:
            n_subs = int(bin_packet[:11], 2)
            bin_packet = bin_packet[11:]

            for idx in range(n_subs):
                sub_value, bin_packet = parse_packet_2(bin_packet)
                values.append(sub_value)

        return operation(type_id, values), bin_packet

In [ ]:
value, _ = parse_packet_2(binary)
value

In [ ]:
puzzle.answer_b = value

## Day 15
https://adventofcode.com/2021/day/15
### Prepare input

In [ ]:
puzzle = Puzzle(year=2021, day=15)
content = puzzle.input_data.split("\n")

In [ ]:
def get_adjacents(idx: int, jdx: int) -> tp.List[tp.Tuple[int, int]]:

    return [(idx - 1, jdx), (idx + 1, jdx), (idx, jdx - 1), (idx, jdx + 1)]

In [ ]:
graph = defaultdict(lambda: defaultdict(lambda: 10))

for y, line in enumerate(content):
    for x, pt in enumerate(line):
        neighbours = get_adjacents(x, y)
        for n in neighbours:
            if 0 <= n[0] < len(content[0]) and 0 <= n[1] < len(content):
                graph[(x, y)][n] = int(content[n[1]][n[0]])

### Part 1

In [ ]:
def dijkstra(graph, start: tp.Tuple[int, int], end):
    unseen_nodes = set(graph.keys())

    shortest_path = defaultdict(list)
    cur_path_risk = defaultdict(lambda: 1000)
    cur_path_risk[start] = 0

    min_node = start
    while unseen_nodes:
        min_node = None
        for node in unseen_nodes:
            if min_node is None:
                min_node = node
            elif cur_path_risk[node] < cur_path_risk[min_node]:
                min_node = node

        if min_node is None:
            return cur_path_risk, shortest_path

        unseen_nodes.remove(min_node)
        cur_risk = cur_path_risk[min_node]

        for neighbour, risk in graph[min_node].items():
            total_risk = cur_risk + risk

            if (
                neighbour not in cur_path_risk.keys()
                or total_risk < cur_path_risk[neighbour]
            ):
                cur_path_risk[neighbour] = total_risk
                shortest_path[neighbour].append(min_node)
                if neighbour == end:
                    return cur_path_risk, shortest_path

In [ ]:
h = len(content)
w = len(content[0])

total_risks, shortest = dijkstra(graph, (0, 0), (w - 1, h - 1))

In [ ]:
total_risks[(w - 1, h - 1)]

### Part 2

In [ ]:
small = np.array([[int(c) for c in line] for line in content])

In [ ]:
big = [small]

for _ in range(4):
    big.append(big[-1] % 9 + 1)

big = [np.concatenate(big, axis=1)]

for _ in range(4):
    big.append(big[-1] % 9 + 1)

big = np.concatenate(big)

In [ ]:
graph2 = defaultdict(lambda: defaultdict(lambda: 10))

h, w = big.shape
for y in range(h):
    for x in range(w):
        neighbours = get_adjacents(x, y)
        for n in neighbours:
            if 0 <= n[0] < w and 0 <= n[1] < h:
                graph2[(x, y)][n] = big[y, x]

In [ ]:
total_risks, shortest = dijkstra(graph2, (0, 0), (w - 1, h - 1))

## Part 2 fortement inspiré chez Johan ^^'' 

In [ ]:
data = big
distances = np.zeros(data.shape)
distances[:, :] = 10 ** 20
distances[0, 0] = 0
visited = set()
to_visit = {(0, 0)}
lattice = set((x, y) for y in range(data.shape[0]) for x in range(data.shape[1]))

In [ ]:
def evaluate(node):
    x, y = node
    for dx, dy in [(-1, 0), (0, 1), (1, 0), (0, -1)]:
        neighbor = x + dx, y + dy
        if neighbor in lattice:
            distances[neighbor] = min(
                distances[node] + data[neighbor], distances[neighbor]
            )
            if neighbor not in visited:
                to_visit.add(neighbor)
    visited.add(node)

In [ ]:
while to_visit:
    node = min([(x, y, distances[x, y]) for x, y in to_visit], key=lambda t: t[2])[:2]
    evaluate(node)
    to_visit = to_visit - visited

In [ ]:
int(distances[-1, -1])

In [ ]:
total_risks[(w - 1, h - 1)]

## Day 14
https://adventofcode.com/2021/day/14
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2021, day=14)
template, content = puzzle.input_data.split("\n\n")

In [ ]:
rules = dict([x.split(" -> ") for x in content.split("\n")])

### Part 1

In [ ]:
def step(in_str: str) -> str:
    new_str = ""
    for c1, c2 in zip(in_str[:-1], in_str[1:]):
        pair = c1 + c2
        insert = rules[pair]
        new_str += c1 + insert
    new_str += in_str[-1]
    return new_str

In [ ]:
def diff(count: Counter) -> int:
    return count.most_common()[0][1] - count.most_common()[-1][1]

In [ ]:
tmp = template
for _ in range(10):
    tmp = step(tmp)

In [ ]:
count = Counter(tmp)
answ = diff(count)
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
def smart_step(in_count: Counter) -> Counter:
    new_count = Counter()
    for pair, c in in_count.items():
        insert = rules[pair]
        new_count[pair[0] + insert] += c
        new_count[insert + pair[1]] += c
    return new_count

In [ ]:
pairs = [c1 + c2 for c1, c2 in zip(template[:-1], template[1:])]
count = Counter(pairs)

for _ in range(40):
    count = smart_step(count)

In [ ]:
char_count = Counter()
for pair, c in count.items():
    outcount[pair[0]] += c
    outcount[pair[1]] += c

In [ ]:
answ = diff(char_count) // 2
answ

In [ ]:
puzzle.answer_b = answ

## Day 13
https://adventofcode.com/2021/day/13
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2021, day=13)
dots, folds = puzzle.input_data.split("\n\n")

In [ ]:
dots = [[int(y) for y in x.split(",")] for x in dots.split("\n")]
dots = set([tuple(d) for d in dots])

In [ ]:
folds = [fold.split(" ")[2].split("=") for fold in folds.split("\n")]

### Part 1

In [ ]:
def fold_paper(dots, fold: tp.Sequence[str]):

    fold_axis = 0 if fold[0] == "x" else 1
    fold_value = int(fold[1])

    new_dots = dots.copy()
    for dot in dots:
        if dot[fold_axis] > fold_value:
            tmp_dot = list(dot)
            tmp_dot[fold_axis] = 2 * fold_value - tmp_dot[fold_axis]
            new_dots.remove(dot)
            new_dots.add(tuple(tmp_dot))

    return new_dots

In [ ]:
new_dots = fold_paper(dots, folds[0])

In [ ]:
answ = len(new_dots)
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
new_dots = dots.copy()
for fold in folds:
    new_dots = fold_paper(new_dots, fold)

In [ ]:
paper_w = max(new_dots, key=lambda x: x[0])[0] + 1
paper_h = max(new_dots, key=lambda x: x[1])[1] + 1

paper = np.zeros((paper_w, paper_h))
for x, y in new_dots:
    paper[x, y] = 1

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(paper.T)

In [ ]:
puzzle.answer_b = "EFJKZLBL"

## Day 12
https://adventofcode.com/2021/day/12
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2021, day=12)
content = [x.split("-") for x in puzzle.input_data.split("\n")]

In [ ]:
graph = defaultdict(set)

for n1, n2 in content:
    graph[n1].add(n2)
    graph[n2].add(n1)

In [ ]:
big_caves = set([node for node in graph.keys() if str.lower(node) != node])
small_caves = set(graph.keys()) - big_caves - {"start", "end"}

### Part 1

In [ ]:
def find_all_paths(graph, cur_path, end, part2=False):

    all_paths = []

    start = cur_path[-1]
    for node in graph[start]:
        counter = Counter(cur_path)
        if (
            node not in cur_path
            or node in big_caves
            or (
                node in small_caves
                and all(counter[sm] <= 1 for sm in small_caves)
                and part2 == True
            )
        ):
            new_path = cur_path.copy()
            new_path.append(node)
            if node == end:
                all_paths.append(new_path)
            else:
                new_paths = find_all_paths(graph, new_path, end, part2)
                all_paths.extend(new_paths)

    return all_paths

In [ ]:
paths = find_all_paths(graph, ["start"], "end")

In [ ]:
answ = len(paths)
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
paths_2 = find_all_paths(graph, ["start"], "end", True)

In [ ]:
answ = len(paths_2)
answ

In [ ]:
puzzle.answer_b = answ

## Day 11
https://adventofcode.com/2021/day/11
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2021, day=11)
content = [[int(char) for char in line] for line in puzzle.input_data.split("\n")]

### Part 1

In [ ]:
from itertools import product

list(product([-1, 0, 1], repeat=2))

In [ ]:
def get_adjacents(idx: int, jdx: int) -> tp.List[tp.Tuple[int, int]]:

    return [
        (idx + dy, jdx + dx)
        for dx, dy in product([-1, 0, 1], repeat=2)
        if dx != 0 or dy != 0
    ]

In [ ]:
def check(n_flashes: int, levels: np.array, point: tp.Tuple[int, int]):

    h, w = levels.shape

    levels[point] = 0
    n_flashes += 1

    adjacents = get_adjacents(point[0], point[1])
    for adj in adjacents:
        if 0 <= adj[0] < h and 0 <= adj[1] < w:
            if levels[adj] not in {0, 10}:
                levels[adj] += 1
                if levels[adj] == 10:
                    n_flashes, levels = check(n_flashes, levels, adj)

    return n_flashes, levels

In [ ]:
def one_step(levels: np.array) -> tp.Tuple[int, np.array]:

    h, w = levels.shape

    n_flashes = 0

    levels += 1

    for y in range(h):
        for x in range(w):
            if levels[y, x] == 10:
                cur_flashes, cur_levels = check(n_flashes, levels, (y, x))
                n_flashes = cur_flashes
                levels = cur_levels

    return n_flashes, levels

In [ ]:
levels = np.array(content)

tot_flashes = 0
for _ in range(100):
    n_flashes, levels = one_step(levels)
    tot_flashes += n_flashes

tot_flashes

In [ ]:
puzzle.answer_a = tot_flashes

### Part 2

In [ ]:
levels = np.array(content)

count = 0
while levels.any():
    _, levels = one_step(levels)
    count += 1

count

In [ ]:
puzzle.answer_b = count

## Day 10
https://adventofcode.com/2021/day/10
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2021, day=10)
content = puzzle.input_data.split("\n")

### Part 1

In [ ]:
char_map = {"(": ")", "[": "]", "{": "}", "<": ">"}

In [ ]:
def check_corrupted(line: str) -> tp.Union[str, None]:
    cur_chunk = deque()
    for char in line:
        if char in char_map.keys():
            cur_chunk.append(char)
        elif char in char_map.values():
            last_char = cur_chunk.pop()
            if char_map[last_char] != char:
                return char

In [ ]:
corrupted = [check_corrupted(line) for line in content]

In [ ]:
char_points = {
    ")": 3,
    "]": 57,
    "}": 1197,
    ">": 25137,
    None: 0,
}

points = sum(char_points[char] for char in corrupted)

In [ ]:
points

In [ ]:
puzzle.answer_a = points

### Part 2

In [ ]:
new_content = set(content) - set([line for _, line, _ in corrupted])

In [ ]:
char_points = {
    ")": 1,
    "]": 2,
    "}": 3,
    ">": 4,
}


def close_open_line(line: str):
    open_seq = deque()
    for char in line:
        if char in char_map.keys():
            open_seq.append(char)
        elif char in char_map.values():
            last_char = open_seq.pop()

    score = 0
    while open_seq:
        cur_char = open_seq.pop()
        closing_char = char_map[cur_char]
        score *= 5
        score += char_points[closing_char]

    return score

In [ ]:
total_scores = [close_open_line(line) for line in new_content]

In [ ]:
total_scores.sort()

In [ ]:
answ = total_scores[len(total_scores) // 2]
answ

In [ ]:
puzzle.answer_b = answ

## Day 9
https://adventofcode.com/2021/day/9
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2021, day=9)
content = puzzle.input_data.split("\n")

In [ ]:
depth = defaultdict(lambda: 10)

for y, line in enumerate(content):
    for x, point in enumerate(line):
        depth[(x, y)] = int(point)

### Part 1

In [ ]:
def get_adjacents(idx: int, jdx: int) -> tp.List[tp.Tuple[int, int]]:
    return [(idx - 1, jdx), (idx + 1, jdx), (idx, jdx - 1), (idx, jdx + 1)]

In [ ]:
# must define since dict changes size during iteration
height = len(content)
width = len(content[0])

low = []

for x in range(width):
    for y in range(height):
        val = depth[(x, y)]

        adjacents = get_adjacents(x, y)

        if all(depth[adj] > val for adj in adjacents):
            low.append((x, y))

In [ ]:
answ = sum(depth[point] + 1 for point in low)
answ

In [ ]:
submit(answ, part="a", day=9, year=2021)

### Part 2

In [ ]:
def get_bassin_size(point: tp.Tuple[int, int], seen: tp.Optional[tp.Set] = None) -> int:

    seen = seen if seen is not None else set()

    x, y = point

    adjacents = get_adjacents(x, y)

    size = 0
    for adj in adjacents:
        if adj not in seen and depth[adj] < 9:
            seen.add(adj)
            size += 1 + get_bassin_size(adj, seen)

    return size

In [ ]:
bassin_sizes = []

for point in low:
    bassin_sizes.append(get_bassin_size(point))

answ = math.prod(sorted(bassin_sizes)[-3:])
answ

In [ ]:
submit(answ, part="b", day=9, year=2021)

## Day 8
https://adventofcode.com/2021/day/8
### Prepare input 

### Part 1

In [ ]:
puzzle = Puzzle(year=2021, day=8)
content = [x.split(" | ") for x in puzzle.input_data.split("\n")]
content = [[x1.split(" "), x2.split(" ")] for x1, x2 in content]

### Part 1

In [ ]:
n_segments = {0: 6, 1: 2, 2: 5, 3: 5, 4: 4, 5: 5, 6: 6, 7: 3, 8: 7, 9: 6}

In [ ]:
answ = 0
for _, output in content:
    for digits in output:
        if len(digits) in {n_segments[1], n_segments[4], n_segments[7], n_segments[8]}:
            answ += 1
answ

In [ ]:
submit(answ, part="a", day=8, year=2021)

### Part2

In [ ]:
def decode(patterns: tp.Sequence[str]) -> tp.Dict[tp.FrozenSet, int]:

    positions = {}
    solutions = {}

    solutions[1] = [el for el in patterns if len(el) == 2][0]
    solutions[7] = [el for el in patterns if len(el) == 3][0]
    solutions[4] = [el for el in patterns if len(el) == 4][0]
    solutions[8] = [el for el in patterns if len(el) == 7][0]

    positions["t"] = set(solutions[7]) - set(solutions[1])

    solutions[9] = [
        el for el in patterns if len(el) == 6 and len(set(el) - set(solutions[4])) == 2
    ][0]
    positions["b"] = set(solutions[9]) - set(solutions[4]) - positions["t"]

    solutions[0] = [
        el
        for el in patterns
        if len(el) == 6 and el != solutions[9] and set(solutions[7]) <= set(el)
    ][0]
    solutions[6] = [
        el
        for el in patterns
        if len(el) == 6 and el != solutions[9] and el != solutions[0]
    ][0]

    positions["m"] = set(solutions[6]) - set(solutions[0])
    positions["tr"] = set(solutions[0]) - set(solutions[6])
    positions["br"] = set(solutions[1]) - positions["tr"]

    solutions[5] = [
        el for el in patterns if len(el) == 5 and not (positions["tr"] <= set(el))
    ][0]
    solutions[3] = [
        el
        for el in patterns
        if len(el) == 5 and el != solutions[5] and positions["br"] <= set(el)
    ][0]
    solutions[2] = [
        el
        for el in patterns
        if len(el) == 5 and el != solutions[5] and el != solutions[3]
    ][0]

    return {frozenset(v): k for k, v in solutions.items()}

In [ ]:
answ = 0
for entry in content:
    patterns, outputs = entry

    code = decode(patterns)

    result = 0
    for out in outputs:
        result = result * 10 + code[frozenset(out)]
    answ += result

answ

In [ ]:
submit(answ, part="b", day=8, year=2021)

## Day 7
https://adventofcode.com/2021/day/7
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2021, day=7)
content = list(map(int, puzzle.input_data.split(",")))

### Part 1

In [ ]:
from statistics import median, mean

In [ ]:
med_pos = int(median(content))


def dist(pos: int) -> int:
    return abs(pos - med_pos)


answ = sum(map(dist, content))
answ

In [ ]:
submit(answ, part="a", day=7, year=2021)

### Part 2

In [ ]:
mean_pos = int(mean(content))


def dist2(pos: int) -> int:
    n = abs(pos - mean_pos)
    return n * (n + 1) // 2


answ = sum(map(dist2, content))
answ

In [ ]:
submit(answ, part="b", day=7, year=2021)

## Day 6
https://adventofcode.com/2021/day/6
### Prepare input

In [ ]:
puzzle = Puzzle(year=2021, day=6)
content = list(map(int, puzzle.input_data.split(",")))

### Part 1

In [ ]:
def iterate(cur_count: Counter) -> Counter:
    new_count = Counter()
    for timer in range(1, 9):
        new_count[timer - 1] = cur_count[timer]
    new_count[6] += cur_count[0]
    new_count[8] += cur_count[0]

    return new_count

In [ ]:
count = Counter(content)
for _ in range(80):
    count = iterate(count)

In [ ]:
answ = sum(count.values())
answ

In [ ]:
submit(answ, part="a", day=6, year=2021)

### Part 2

In [ ]:
count = Counter(content)
for _ in range(256):
    count = iterate(count)

answ = sum(count.values())
answ

In [ ]:
submit(answ, part="b", day=6, year=2021)

## Day 5
https://adventofcode.com/2021/day/5
### Prepare input

In [ ]:
puzzle = Puzzle(year=2021, day=5)

In [ ]:
content = puzzle.input_data.split("\n")

In [ ]:
class Diagram:
    def __init__(self, incontent: tp.Sequence[str]) -> None:

        self.segments = [
            [tuple(map(int, row.split(","))) for row in line.split(" -> ")]
            for line in incontent
        ]

        self.rows = self.get_rows()
        self.cols = self.get_cols()
        self.diags = self.get_diags()

        self.counter = Counter()

    def get_rows(self) -> tp.List[tp.List[tp.Tuple[int, int]]]:
        return [segm for segm in self.segments if segm[0][0] == segm[1][0]]

    def get_cols(self) -> tp.List[tp.List[tp.Tuple[int, int]]]:
        return [segm for segm in self.segments if segm[0][1] == segm[1][1]]

    def get_diags(self) -> tp.List[tp.List[tp.Tuple[int, int]]]:
        return [
            segm
            for segm in self.segments
            if segm[0][0] != segm[1][0] and segm[0][1] != segm[1][1]
        ]

    def count_rows(self) -> None:
        for (x1, y1), (x2, y2) in self.rows:
            dy = 1 if y1 < y2 else -1
            self.counter.update([(x1, y) for y in range(y1, y2 + dy, dy)])

    def count_cols(self) -> None:
        for (x1, y1), (x2, y2) in self.cols:
            dx = 1 if x1 < x2 else -1
            self.counter.update([(x, y1) for x in range(x1, x2 + dx, dx)])

    def count_diags(self) -> None:
        for (x1, y1), (x2, y2) in self.diags:
            dx = 1 if x1 < x2 else -1
            dy = 1 if y1 < y2 else -1
            self.counter.update(
                [(x, y) for x, y in zip(range(x1, x2 + dx, dx), range(y1, y2 + dy, dy))]
            )

    def overlap(self, part: int = 1) -> int:
        self.reset_counter()

        self.count_rows()
        self.count_cols()
        if part == 2:
            self.count_diags()

        return len([point for point, count in self.counter.items() if count >= 2])

    def reset_counter(self) -> None:
        self.counter = Counter()

### Part 1

In [ ]:
diagram = Diagram(content)

In [ ]:
answ = diagram.overlap()
answ

In [ ]:
submit(answ, part="a", day=5, year=2021)

### Part 2

In [ ]:
answ = diagram.overlap(2)
answ

In [ ]:
submit(answ, part="b", day=5, year=2021)

## Day 4
https://adventofcode.com/2021/day/4
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2021, day=4)

In [ ]:
content = puzzle.input_data.split("\n\n")

In [ ]:
numbers = [int(x) for x in content[0].split(",")]

In [ ]:
class Board:
    def __init__(self, inboard: tp.Sequence[tp.Sequence[int]]) -> None:
        self.board = np.array(inboard)
        self.rows = [set(r) for r in self.board]
        self.cols = [set(c) for c in np.transpose(self.board)]

    def sum_leftover(self) -> int:
        return sum(sum(r) for r in self.rows)

    def discard_nb(self, nb: int) -> None:
        self.rows = [r - {nb} for r in self.rows]
        self.cols = [c - {nb} for c in self.cols]

    def is_winner(self) -> bool:
        return not (all(self.rows) and all(self.cols))

In [ ]:
def set_boards(boards_content: tp.Sequence[str]) -> tp.List[Board]:
    return [
        Board([[int(n) for n in row.split()] for row in line.split("\n")])
        for line in boards_content
    ]

In [ ]:
boards = set_boards(content[1:])

success_rate = []
for bdx, board in enumerate(boards):
    for ndx, nb in enumerate(numbers):
        board.discard_nb(nb)
        if board.is_winner():
            success_rate.append([ndx, nb * board.sum_leftover()])
            break

success_rate.sort(key=lambda x: x[0])

### Part 1

In [ ]:
answ = success_rate[0][1]
answ

In [ ]:
submit(answ, part="a", day=4, year=2021)

### Part 2

In [ ]:
answ = success_rate[-1][1]
answ

In [ ]:
submit(answ, part="b", day=4, year=2021)

## Day 3
https://adventofcode.com/2021/day/3
### Prepare input

In [ ]:
puzzle = Puzzle(year=2021, day=3)
content = puzzle.input_data.split("\n")

In [ ]:
content

### Part 1

In [ ]:
def transpose(inlist: tp.Sequence[str]) -> tp.List[str]:
    return ["".join(s) for s in zip(*inlist)]

In [ ]:
def bin2int(binary: tp.Union[str, tp.Sequence[str]]) -> int:
    if isinstance(binary, list):
        return int("".join(binary), 2)
    return int(binary, 2)

In [ ]:
counters = [Counter(pos) for pos in transpose(content)]

In [ ]:
gamma = ["0" if c["0"] > c["1"] else "1" for c in counters]
eps = "".join(str(1 - int(x)) for x in gamma)

In [ ]:
answ = bin2int(gamma) * bin2int(eps)
answ

In [ ]:
submit(answ, part="a", day=3, year=2021)

### Part 2

In [ ]:
def keep(inlist: tp.Sequence[str], bpos: int, bval: str) -> tp.List[str]:
    return [el for el in inlist if el[bpos] == bval]

In [ ]:
def apply_criteria(init: tp.Sequence[str], default: int) -> int:
    out = init.copy()
    idx = 0
    while len(out) > 1:
        counter = Counter(transpose(out)[idx])
        if counter["0"] > counter["1"]:
            out = keep(out, idx, str(1 - default))
        else:
            out = keep(out, idx, str(default))
        idx += 1
    return bin2int(out[0])

In [ ]:
oxygen = apply_criteria(content, 1)
co2 = apply_criteria(content, 0)
anws = oxygen * co2
answ

In [ ]:
submit(answ, part="b", day=3, year=2021)

## Day 2
https://adventofcode.com/2021/day/2
### Prepare input

In [ ]:
puzzle = Puzzle(year=2021, day=2)

In [ ]:
content = [x.split(" ") for x in puzzle.input_data.split("\n")]

In [ ]:
content

### Part 1

In [ ]:
h_pos = 0
depth = 0
for step, x in content:
    x = int(x)

    if step == "forward":
        h_pos += x
    elif step == "down":
        depth += x
    elif step == "up":
        depth -= x

answ = h_pos * depth
answ

In [ ]:
submit(answ, part="a", day=2, year=2021)

### Part 2

In [ ]:
h_pos = 0
depth = 0
aim = 0
for step, x in content:
    x = int(x)

    if step == "forward":
        h_pos += x
        depth += aim * x
    elif step == "down":
        aim += x
    elif step == "up":
        aim -= x

print(h_pos, depth, aim)
answ = h_pos * depth
answ

In [ ]:
submit(answ, part="b", day=2, year=2021)

## Day 1
https://adventofcode.com/2021/day/1

### Prepare input

In [ ]:
puzzle = Puzzle(year=2021, day=1)

In [ ]:
content = [int(x) for x in puzzle.input_data.split("\n")]

### Part 1

In [ ]:
def count_increase(inlist: tp.Sequence[int]) -> int:
    count = 0
    for i in range(len(inlist) - 1):
        if inlist[i] < inlist[i + 1]:
            count += 1

    return count

In [ ]:
answ = count_increase(content)
answ

In [ ]:
submit(answ, part="a", day=1, year=2021)

### Part 2

In [ ]:
sums = [sum(content[i : i + 3]) for i in range(len(content) - 2)]

In [ ]:
answ = count_increase(sums)
answ

In [ ]:
submit(answ, part="b", day=1, year=2021)